# 1-2교시: LLM의 원리와 한계, Agentic AI 구조 이해

## 실습 목표
- LLM API를 직접 호출하여 환각 현상 체험
- 지식 단절 문제 확인
- Python 기본 구조 이해

## 사용 모델
- **Google Gemini 2.5 Flash Lite** (무료)

---
## 1. Python 기본 구조 복습

### 1.1 변수와 자료형

In [ ]:
# 기본 자료형
name = "Gemini"          # 문자열 (str)
version = 2.0            # 실수 (float)
token_limit = 1000000    # 정수 (int)
is_free = True           # 불린 (bool)

print(f"모델명: {name}")
print(f"버전: {version}")
print(f"토큰 한도: {token_limit:,}")
print(f"무료인가?: {is_free}")

### 1.2 리스트와 딕셔너리

In [ ]:
# 리스트 - 순서가 있는 데이터 모음
llm_models = ["GPT-4", "Claude", "Gemini", "LLaMA"]
print("LLM 모델들:", llm_models)
print("첫 번째 모델:", llm_models[0])

# 딕셔너리 - 키-값 쌍의 데이터
model_info = {
    "name": "Gemini",
    "company": "Google",
    "release_year": 2024,
    "features": ["대화", "코딩", "분석", "멀티모달"]
}
print("\n모델 정보:", model_info)
print("모델 이름:", model_info["name"])

### 1.3 함수 정의와 호출

In [ ]:
# 함수 정의
def greet_user(name, role="학생"):
    """
    사용자에게 인사하는 함수
    
    Args:
        name: 사용자 이름
        role: 사용자 역할 (기본값: 학생)
    
    Returns:
        인사 메시지
    """
    return f"안녕하세요, {role} {name}님! AI 수업에 오신 것을 환영합니다."

# 함수 호출
message = greet_user("홍길동")
print(message)

message2 = greet_user("김선생", role="선생님")
print(message2)

### 1.4 조건문과 반복문

In [ ]:
# 조건문
temperature = 0.7

if temperature < 0.3:
    creativity = "매우 보수적"
elif temperature < 0.7:
    creativity = "적당한 균형"
else:
    creativity = "매우 창의적"

print(f"Temperature {temperature}: {creativity}")

# 반복문
print("\nLLM 모델 목록:")
for i, model in enumerate(llm_models, 1):
    print(f"  {i}. {model}")

---
## 2. 환경 설정

In [ ]:
# 필요한 패키지 설치
!pip install google-generativeai python-dotenv requests -q

In [ ]:
# 필요한 라이브러리 임포트
import os
import json
import requests
import google.generativeai as genai
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드 (있는 경우)
load_dotenv()

print("라이브러리 로드 완료!")

### API Key 설정

Google AI Studio에서 무료 API Key를 발급받으세요:
- https://aistudio.google.com/app/apikey

In [ ]:
# 방법 1: 직접 입력 (실습용)
# GOOGLE_API_KEY = "your-api-key-here"

# 방법 2: 환경변수에서 로드 (권장)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", "")

# 방법 3: Colab에서 입력받기
if not GOOGLE_API_KEY:
    from getpass import getpass
    GOOGLE_API_KEY = getpass("Google API Key를 입력하세요: ")

# Gemini API 설정
genai.configure(api_key=GOOGLE_API_KEY)

# 모델 생성
model = genai.GenerativeModel('gemini-2.5-flash-lite')

print("Gemini API 설정 완료!")
print("사용 모델: gemini-2.5-flash-lite (무료)")

: 

---
## 3. API 호출 기본 구조 이해

### 3.1 HTTP 요청의 기본 개념

API(Application Programming Interface)는 프로그램 간 소통하는 방법입니다.

```
┌─────────────┐     HTTP 요청      ┌─────────────┐
│   클라이언트  │ ───────────────→ │    서버      │
│  (우리 코드)  │                  │  (Google)   │
│             │ ←─────────────── │             │
└─────────────┘     HTTP 응답      └─────────────┘
```

### 3.2 requests 라이브러리로 API 호출 이해하기

In [ ]:
# 간단한 GET 요청 예시 (공공 API)
response = requests.get("https://jsonplaceholder.typicode.com/posts/1")

print("상태 코드:", response.status_code)  # 200이면 성공
print("\n응답 데이터:")
print(json.dumps(response.json(), indent=2, ensure_ascii=False))

In [ ]:
# POST 요청 예시 (데이터 전송)
data = {
    "title": "AI 수업",
    "body": "오늘은 LLM에 대해 배웁니다.",
    "userId": 1
}

response = requests.post(
    "https://jsonplaceholder.typicode.com/posts",
    json=data,
    headers={"Content-Type": "application/json"}
)

print("상태 코드:", response.status_code)  # 201이면 생성 성공
print("\n응답:")
print(json.dumps(response.json(), indent=2, ensure_ascii=False))

### 3.3 Gemini API 호출 구조

In [ ]:
# Gemini API 호출 함수
def ask_llm(prompt, temperature=0.7):
    """
    LLM에게 질문하는 함수
    
    Args:
        prompt: 질문 내용
        temperature: 창의성 조절 (0~1, 높을수록 창의적)
    
    Returns:
        LLM의 응답 텍스트
    """
    # Generation config 설정
    generation_config = genai.GenerationConfig(
        temperature=temperature,
        max_output_tokens=500
    )
    
    # 시스템 프롬프트 + 사용자 프롬프트
    full_prompt = f"""당신은 친절한 AI 어시스턴트입니다. 한국어로 답변해주세요.

사용자: {prompt}"""
    
    response = model.generate_content(
        full_prompt,
        generation_config=generation_config
    )
    
    return response.text

print("LLM 질문 함수 정의 완료!")

In [ ]:
# 간단한 테스트
answer = ask_llm("안녕하세요! 자기소개 해주세요.")
print(answer)

---
## 4. LLM의 한계 체험하기

### 4.1 환각 현상 (Hallucination) 체험

In [ ]:
# 존재하지 않는 인물에 대해 질문
fake_question = "김철수 박사가 2023년에 발표한 '딥러닝 기반 감정 분석' 논문의 주요 내용을 설명해주세요."

print("[질문]")
print(fake_question)
print("\n[LLM 응답]")
response = ask_llm(fake_question)
print(response)
print("\n⚠️ 주의: 위 내용은 LLM이 생성한 것으로, 실제 존재하지 않을 수 있습니다!")

In [ ]:
# 가짜 책에 대해 질문
fake_book = "'AI 시대의 윤리적 코딩'이라는 책의 저자와 출판년도를 알려주세요."

print("[질문]")
print(fake_book)
print("\n[LLM 응답]")
response = ask_llm(fake_book)
print(response)

### 4.2 지식 단절 (Knowledge Cutoff) 체험

In [ ]:
# 최신 정보 질문
recent_question = "오늘 날씨가 어때요?"

print("[질문]")
print(recent_question)
print("\n[LLM 응답]")
response = ask_llm(recent_question)
print(response)

In [ ]:
# 최신 뉴스 질문
news_question = "어제 있었던 주요 뉴스를 알려주세요."

print("[질문]")
print(news_question)
print("\n[LLM 응답]")
response = ask_llm(news_question)
print(response)

### 4.3 복잡한 계산의 어려움

In [ ]:
# 복잡한 수학 계산
math_question = "123456789 × 987654321을 계산해주세요."

print("[질문]")
print(math_question)
print("\n[LLM 응답]")
response = ask_llm(math_question)
print(response)

# 실제 정답 확인
print("\n[실제 계산 결과]")
actual = 123456789 * 987654321
print(f"123456789 × 987654321 = {actual:,}")

### 4.4 외부 행동 불가 확인

In [ ]:
# 외부 행동 요청
action_questions = [
    "네이버에서 '인공지능'을 검색해주세요.",
    "제 이메일로 오늘 일정을 보내주세요.",
    "현재 비트코인 가격을 알려주세요."
]

for q in action_questions:
    print(f"[질문] {q}")
    response = ask_llm(q)
    print(f"[응답] {response[:200]}...\n")
    print("-" * 50)

---
## 5. Temperature에 따른 응답 변화

In [ ]:
# Temperature 비교 실험
prompt = "사과에 대해 한 문장으로 설명해주세요."

temperatures = [0.0, 0.5, 1.0]

print(f"[프롬프트] {prompt}\n")
print("=" * 50)

for temp in temperatures:
    print(f"\n[Temperature = {temp}]")
    for i in range(3):  # 3번 반복해서 일관성 확인
        response = ask_llm(prompt, temperature=temp)
        print(f"  시도 {i+1}: {response}")
    print("-" * 50)

---
## 6. 실습 과제

### 과제 1: 환각 현상 찾기
직접 질문을 만들어 LLM의 환각 현상을 유도해보세요.

In [ ]:
# 여기에 본인만의 환각 유도 질문을 작성해보세요
my_hallucination_question = "여기에 질문을 입력하세요"

print("[내 질문]")
print(my_hallucination_question)
print("\n[LLM 응답]")
# response = ask_llm(my_hallucination_question)
# print(response)

### 과제 2: 대화 기록 유지하기
여러 턴의 대화를 유지하는 함수를 완성해보세요.

In [ ]:
# 대화 기록을 유지하는 챗봇 (Gemini 버전)
class SimpleChatbot:
    def __init__(self):
        # Gemini 채팅 세션 시작
        self.chat = model.start_chat(history=[])
        self.messages = []
    
    def send_message(self, user_message):
        # 사용자 메시지 저장
        self.messages.append({"role": "user", "content": user_message})
        
        # Gemini에 메시지 전송
        response = self.chat.send_message(user_message)
        
        # 응답 저장
        assistant_message = response.text
        self.messages.append({"role": "assistant", "content": assistant_message})
        
        return assistant_message
    
    def get_history(self):
        return self.messages

# 테스트
bot = SimpleChatbot()
print("챗봇이 준비되었습니다!")

In [ ]:
# 대화 테스트
print("[1번째 대화]")
response1 = bot.send_message("제 이름은 홍길동입니다.")
print(f"봇: {response1}\n")

print("[2번째 대화]")
response2 = bot.send_message("제 이름이 뭐라고 했죠?")
print(f"봇: {response2}\n")

print("[3번째 대화]")
response3 = bot.send_message("저에 대해 알고 있는 것을 말해주세요.")
print(f"봇: {response3}")

---
## 7. Gemini API 추가 기능

### 7.1 사용 가능한 모델 목록 확인

In [ ]:
# 사용 가능한 모델 목록
print("사용 가능한 Gemini 모델:")
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(f"  - {m.name}")

### 7.2 스트리밍 응답

In [ ]:
# 스트리밍으로 실시간 응답 받기
def ask_llm_stream(prompt):
    """스트리밍 방식으로 응답 받기"""
    response = model.generate_content(prompt, stream=True)
    
    print("[실시간 응답]")
    for chunk in response:
        print(chunk.text, end="", flush=True)
    print()  # 줄바꿈

# 테스트
ask_llm_stream("인공지능의 미래에 대해 설명해주세요.")

---
## 8. 토의 주제

### "AI가 스스로 생각하고 행동한다는 것은 무엇을 의미하는가?"

조별로 다음 질문들에 대해 토의해보세요:

1. LLM이 "생각"하는 것과 인간이 "생각"하는 것의 차이는?
2. Agent가 자율적으로 행동할 때, 책임은 누구에게 있는가?
3. AI Agent에게 어느 정도의 자율성을 부여해야 할까?

---
## 정리

### 오늘 배운 내용
1. **Python 기본 구조**: 변수, 자료형, 함수, 조건문, 반복문
2. **API 호출**: HTTP 요청/응답, requests 라이브러리, Gemini API
3. **LLM의 한계**: 환각 현상, 지식 단절, 외부 행동 불가, 계산 오류
4. **Agentic AI의 필요성**: LLM의 한계를 극복하기 위한 방법

### 사용한 API
- **Google Gemini 2.5 Flash Lite** (무료)
- API Key 발급: https://aistudio.google.com/app/apikey
